In [92]:
def getLetter():
    input_letter = input()
    if input_letter.isalpha() and len(input_letter) == 1:
        return input_letter.lower()
    else:
        print('Error, try again')
        return getLetter()

In [93]:
class Letter():
    def __init__(self, letter, open_flag):
        self.letter = letter
        self.open_flag = open_flag
    
    def show(self):
        if self.open_flag:
            return self.letter
        else:
            return '*'

In [98]:
class Slovo():
    def __init__(self, word, attemts_number, is_letter_in_word=False):
        self.word = word
        self.attemts_number = attemts_number
        self.is_letter_in_word = is_letter_in_word

        letter_list = []
        for i in word:
            letter_list.append(Letter(i, False))
        self.letter_list = letter_list

        inputs_list = []
        self.inputs_list = inputs_list

    def check(self, input_letter):
        if input_letter not in self.inputs_list:
            self.inputs_list.append(input_letter)
        else:
            print(f'You have already entered the letter "{input_letter}"')
            self.is_letter_in_word = True
        
    def decrease_attemts(self, input_letter):
        if not self.is_letter_in_word:
            print(f'The letter "{input_letter}" is not in the word')
            self.attemts_number -=1

    def show(self, show_word_flag, new_word=None, input_letter=None):
        if show_word_flag:
            new_word = self.word
            return new_word
        else:
            self.is_letter_in_word = False
            for i in self.letter_list:
                if i.letter == input_letter:
                    self.is_letter_in_word = True
                    i.open_flag = True
                new_word += i.show()
            return new_word

    def game_mechanics(self, show_word_flag, input_letter):
        new_word = ''
        new_word = self.show(show_word_flag, new_word, input_letter)
        self.check(input_letter)
        self.decrease_attemts(input_letter)
                
        return (new_word, self.attemts_number)
            


In [99]:
def game(word, attemts_number):
    slovo = Slovo(word, attemts_number)
    while attemts_number != 0:
        input_letter = getLetter()
        new_word, attemts_number = slovo.game_mechanics(False, input_letter)
        print(f'{new_word} \nAttempts left: {attemts_number}')

        if new_word == slovo.show(True):
            print('You win')
            break
    if attemts_number == 0:  
        print('Game over')

In [100]:
game('language', 7)

l******* 
Attempts left: 7
The letter "y" is not in the word
l******* 
Attempts left: 6
la***a** 
Attempts left: 6
The letter "r" is not in the word
la***a** 
Attempts left: 5
lan**a** 
Attempts left: 5
lan**a*e 
Attempts left: 5
The letter "i" is not in the word
lan**a*e 
Attempts left: 4
lang*age 
Attempts left: 4
language 
Attempts left: 4
You win
